In [1]:
from transformers import pipeline
import pandas as pd
import torch

In [2]:
# Load the data
file_path = "..\\..\\Data\processed\Intents\\final_dataset.xlsx"
data = pd.read_excel(file_path)

data

,user_id,conversations,company_name,cleaned_conversations,structured_conversation,entities
0,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[{'Company_name': 'sprintcare'}, {'conversatio...","{'products': [], 'services': ['customer servic..."
1,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[{'Company_name': 'sprintcare'}, {'conversatio...","{'products': [], 'services': ['Customer Servic..."
2,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[{'Company_name': 'sprintcare'}, {'conversatio...","{'products': [], 'services': ['Customer Servic..."
3,115712,Customer: @115714 @sprintcare how can I get in...,sprintcare,Customer how can I get in touch with someone t...,"[{'Company_name': 'sprintcare'}, {'conversatio...","{'products': ['Spring Corporate'], 'services':..."
4,115712,Customer: @115714 @sprintcare how can I get in...,sprintcare,Customer how can I get in touch with someone t...,"[{'Company_name': 'sprintcare'}, {'conversatio...","{'products': ['Spring Corporate'], 'services':..."
...,...,...,...,...,...,...
4995,115766,"Customer: In two days, Call of Duty® returns t...",ATVIAssist,Customer In two days Call of Duty returns to W...,"[{'Company_name': 'ATVIAssist'}, {'conversatio...","{'products': ['Call of Duty', 'World War II', ..."
4996,115766,Customer: JAPAN: It's time to jump in! #CODWWI...,ATVIAssist,Customer JAPAN Its time to jump in CODWWII \nC...,"[{'Company_name': 'ATVIAssist'}, {'conversatio...","{'products': ['CODWWII'], 'services': ['Digita..."
4997,115766,Customer: JAPAN: It's time to jump in! #CODWWI...,ATVIAssist,Customer JAPAN Its time to jump in CODWWII \nC...,"[{'Company_name': 'ATVIAssist'}, {'conversatio...","{'products': ['CODWWII'], 'services': ['Call o..."
4998,115766,Customer: JAPAN: It's time to jump in! #CODWWI...,ATVIAssist,Customer JAPAN Its time to jump in CODWWII \nC...,"[{'Company_name': 'ATVIAssist'}, {'conversatio...","{'products': ['CODWWII', 'Steam'], 'services':..."


In [3]:
# Extract the structured conversations and intents
structured_conversations = data['structured_conversation']
entities = data['entities']

In [5]:
# Load the Llama pipeline
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
def extract_relationships(conversation_text, entities):
    """
    Extract relationships between entities from structured conversation text.
    """
    system_prompt = """
    Step 1: Model Entity 
    You are LLAMA_3.2_3B, a relationship extraction model.

    Step 2: Task  
    Identify meaningful relationships from the provided structured conversations.

    Step 3: Common Relationships 
    1. hasIssue: product/service → issue  
    2. appliesTo: issue → intent  
    3. resolvesWith: issue → resolution  
    4. providedBy: product/service → provider  

    Step 4: Output Format
    Return your findings as a JSON array of RDF triples:  
    ```
    [
        {"subject": "<entity>", "predicate": "<relationship>", "object": "<entity_or_intent>"}
    ]
    ```

    Step 5: Strict Rules  
    - ONLY output the JSON array.  
    - NO extra text before or after the array.  
    - If there is **no product**, do **not** create triples with a product.  
    - If there is **no issue**, do **not** create triples with an issue.  
    - If there is **no intent**, do **not** create triples with an intent.
    - DO NOT WRITE ANY TEXT OTHER THAN OUTPUT FORMAT.
    - DO NOT WRITE ANY TEXT BEFORE OR AFTER THE OUTPUT FORMAT.
    """

    # Create messages for the pipeline
    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": f"""
            Here is the conversation text: '{conversation_text}'.
            Extracted entities: {entities}.
            Identify relationships between these elements and provide RDF triples.
            """,
        },
    ]
    
    # Generate relationships using Llama pipeline
    output = pipe(messages, max_new_tokens=256)

    # Extract and return the generated content
    return output[0]["generated_text"][-1].get("content", "") if output else "No relationships found."


In [17]:
# get 5 random rows and add the relationships to the respected row and after that store the results with only respected rows in it

# Get 5 random rows
sample = data.sample(5)

# Extract relationships for each row
relationships = []

for index, row in sample.iterrows():
    conversation_text = row['structured_conversation']
    entities = row['entities']
    relationships.append(extract_relationships(conversation_text, entities))

# Add the relationships to the sample
sample['relationships'] = relationships

# Store the results
sample.to_excel("..\\..\\Data\processed\\relationships\\final_dataset_with_relationships.xlsx", index=False)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
